# Not Currently Functional

In [362]:
import numpy as np
from scipy import sparse
from igraph import *
from functools import reduce
import copy

In [363]:
#import pip
#pip.main(['install','--upgrade','python-igraph'])

In [364]:
def FastIntesectingQ(ar1,ar2):
    return np.array(np.intersect1d(ar1, ar2, assume_unique=True).size,np.bool).tolist()

def Union(*args):
    return reduce(np.union1d, args)

def DeepUnion(*arrays):
    actualarrays=tuple(np.array(ar,np.uint) for ar in tuple(arrays))
    return np.unique(np.hstack(actualarrays).astype(np.uint))

DeepUnion([3],[6],[])

array([3, 6], dtype=uint32)

In [396]:
def UpgradeGraphForAnalysis(g):
    sorting=np.array(g.topological_sorting('out'))
    names=np.array(g.vs["name"])
    g.contract_vertices(np.argsort(sorting).tolist()); #MutableForm of permute
    #print(names[sorting])
    g.vs["name"]=names[sorting].tolist()
    g.vs["index"]=range(g.vcount())
    g.vs["parents"]=g.get_adjlist('in');
    g.vs["children"]=g.get_adjlist('out');
    g.vs["ancestors"]=[g.subcomponent(i,'in') for i in g.vs]
    g.vs["descendants"]=[g.subcomponent(i,'out') for i in g.vs]
    return 
    

In [483]:
def SetAncestors(g,*args):
    predeepunion=tuple(g.subcomponent(i,'in') for i in DeepUnion(*args))
    return DeepUnion(*predeepunion)

#def SetAncestors(g,*args):
#    return DeepUnion(g.vs[DeepUnion(args).tolist()]["ancestors"])

def AncestralSubgraph(g,*args):
    return g.induced_subgraph(SetAncestors(g,*args))

def ConditionalGraph(g,x,y,z):
    ancgraph=copy.deepcopy(g)
    ancgraph.delete_edges(ancgraph.es.select(_source_notin = SetAncestors(g,x,y,z)))
    #ancgraph=AncestralSubgraph(g,x,y,z);
    flatz=np.atleast_1d(z)
    print(flatz)
    nodecount=g.vcount()
    g.add_vertices(len(flatz))
    print(ancgraph.summary(1).splitlines()[-1])
    for i in flatz:
        print("hello")
        edges_to_add=[(i+nodecount,p) for p in g.neighbors(i, mode='IN')]
        edges_to_delete=[(p,i) for p in g.neighbors(i, mode='IN')]
        ancgraph.add_edges(edges_to_add)
        ancgraph.delete_edges(edges_to_delete)
        edges_to_delete=[(i,c) for c in g.neighbors(i, mode='OUT')]
        ancgraph.delete_edges(edges_to_delete)
    print(ancgraph.summary(1).splitlines())
    Graph.to_undirected(ancgraph)
    print(ancgraph.summary(1).splitlines()[-1])
    return ancgraph

def DSeperated(g,X,Y,Z):
    #g2=ConditionalGraph(g,X,Y,Z).as_undirected() #Doesn't really work. Maybe this messes up the indices?
    g2=Graph.TupleList(ConditionalGraph(g,X,Y,Z).as_undirected().get_edgelist()) #So we are forced to manually compensate
    print(g2.summary(1))
    #g2=Graph.TupleList(ConditionalGraph(g,X,Y,Z).get_edgelist()) #So we are forced to manually compensate
    print(g2.get_edgelist())
    print(g2.vs.indices)
    apparently_seperated = True
    for x in np.atleast_1d(X).flat:
        for y in np.atleast_1d(Y).flat:
            if g2.are_connected(x,y):
                apparently_seperated=False
                return False
    return apparently_seperated


In [485]:
g=Graph.Formula("U3->A:C:D,U2->B:C:D,U1->A:B,A->C,B->D");
#print(g.vs["name"])
#print(g.topological_sorting('out'))
UpgradeGraphForAnalysis(g);
#print(g.vs["name"])
#print(g.topological_sorting('out'))
DSeperated(g,[3],[6],[2])

[2]
U3->A, U3->C, U3->D, A->C, U2->C, U2->D, U2->B, B->D, U1->A, U1->B
hello
['IGRAPH DN-- 7 8 -- ', '+ attr: ancestors (v), children (v), descendants (v), index (v), name (v), parents (v)', '+ edges (vertex names):', 'U3->A, U3->C, U3->D, A->C, U2->C, U2->D, U2->B, B->D']
 D -- U3, U2, B
IGRAPH UN-- 6 8 -- 
+ attr: name (v)
+ edges (vertex names):
0 -- 3, 5, 6
3 -- 0, 5
1 -- 4, 5, 6
4 -- 1, 6
5 -- 0, 3, 1
6 -- 0, 1, 4
[(0, 1), (2, 3), (0, 4), (2, 4), (1, 4), (0, 5), (2, 5), (3, 5)]
[0, 1, 2, 3, 4, 5]


InternalError: Error at c:\projects\python-igraph-jst2e\vendor\build\igraph\igraph-0.8.3-msvc\src\basic_query.c:57: are connected, Invalid vertex id

In [451]:
Graph.TupleList([(0, 3), (2, 3), (1, 4), (2, 4), (0, 5), (1, 5), (3, 5), (0, 6), (1, 6), (4, 6)]).are_connected(3,6)

True

In [452]:
Graph.Adjacency(sparse.lil_matrix(ConditionalGraph(g,[3],[6],[]).get_adjacency_sparse()).toarray().tolist(),mode='UNDIRECTED')

[]
U3->A, U3->C, U3->D, A->C, U2->C, U2->D, U2->B, B->D, U1->A, U1->B
['IGRAPH DN-- 7 10 -- ', '+ attr: ancestors (v), children (v), descendants (v), index (v), name (v), parents (v)', '+ edges (vertex names):', 'U3->A, U3->C, U3->D, A->C, U2->C, U2->D, U2->B, B->D, U1->A, U1->B']
 D -- U3, U2, B


In [370]:
#TOWARD IMPLEMENTING D-SEPERATION TEST
#induced_subgraph
#is_connected('WEAK')
#as_undirected()
#subcomponent(v,'ALL') #for a node
#components('WEAK')
#are_connected(v1,v2)
#cohesive_blocks
#get_automorphisms_vf2